# Python과 DataBase 연동

1. 해당 라이브러리 : DB 접속 드라이버, cx_Oracle, python-oracledb
  - 설치 : Anaconda prompt 창(관리자 권한 실행) pip install 설치할 모듈명
  - 주피터 노트북안에서 설치 : !pip install 설치할 모듈명
  - Anaconda : pip install ~ or conda install ~
  - pip install oracledb --upgrade 최근 버전(권장)
  - pip install cx_Oracle --upgrade
  
2. 접속정보 확인
  - 사용자 계정과 비밀번호 : scott/tiger
  - 오라클 url : localhost:1521/xe

In [1]:
# 주피터 노트북에 라이브러리 설치
#!pip install cx_Oracle --upgrade 

# DB 연동 작업

In [2]:
# 라이브러리(모듈) 연결
import cx_Oracle as cx

## connection : 연결 통로 구축

In [3]:
# db연동 connect("사용자계정", "비밀번호", "url")
# url - localhost(127.0.0.1):1521/xe
conn = cx.connect("scott", "1234", "localhost:1521/xe")
# conn = cx.connect("scott/1234@localhost:1521/xe")

## DB에 테이블 조회

In [13]:
# cursor(커서) : 레코드 셋을 탐색하는 포인터
## 커서 객체 생성
cur = conn.cursor()

## query(질의문)
sql = "select * from product"

## 커서 객체를 이용하여 sql 실행
cur.execute(sql)

for row in cur:
    print(row)
    # clob 자료형은 그냥읽어올 수 없다.
    description = row[3].read()
    print(description)

(1, '레몬', 1500, <cx_Oracle.LOB object at 0x000001FE160EDC90>, 'lemon.jpg')
맛있는 레몬이 왔어요
(2, '오렌지', 2500, <cx_Oracle.LOB object at 0x000001FE160ED510>, 'orange.jpg')
상큼한 오렌지 있어요
(3, '키위', 1000, <cx_Oracle.LOB object at 0x000001FE1605C480>, 'kiwi.jpg')
먹어도 키가 안크는 키위가 있어요
(4, '포도', 5000, <cx_Oracle.LOB object at 0x000001FE160EDC90>, 'grape.jpg')
포도가 맛있다
(5, '딸기', 8000, <cx_Oracle.LOB object at 0x000001FE1605CC00>, 'strawberry.jpg')
딸기도 좋아요
(6, '귤', 500, <cx_Oracle.LOB object at 0x000001FE1605C480>, 'tangerine.jpg')
제주도 대신 귤


## 전체 레코드 삭제 후 테이블에 레코드 삽입하는 작업

In [6]:
sql = "delete from product"
cur.execute(sql)
conn.commit()

In [8]:
# 파이썬에서 DB에 레코드 삽입
items = [
    (1, '레몬', 1500, '맛있는 레몬이 왔어요', 'lemon.jpg'),
    (2, '오렌지', 2500, '상큼한 오렌지 있어요', 'orange.jpg'),
    (3, '키위', 1000, '먹어도 키가 안크는 키위가 있어요', 'kiwi.jpg'),
    (4, '포도', 5000, '포도가 맛있다', 'grape.jpg'),
    (5, '딸기', 8000, '딸기도 좋아요', 'strawberry.jpg'),
    (6, '귤', 500, '제주도 대신 귤', 'tangerine.jpg')
]

for row in items:
    # : 필드인덱스(1부터)
    sql = "insert into product values (:1, :2, :3, :4, :5)"
    print(row)
    cur.execute(sql, row)
    
conn.commit()

(1, '레몬', 1500, '맛있는 레몬이 왔어요', 'lemon.jpg')
(2, '오렌지', 2500, '상큼한 오렌지 있어요', 'orange.jpg')
(3, '키위', 1000, '먹어도 키가 안크는 키위가 있어요', 'kiwi.jpg')
(4, '포도', 5000, '포도가 맛있다', 'grape.jpg')
(5, '딸기', 8000, '딸기도 좋아요', 'strawberry.jpg')
(6, '귤', 500, '제주도 대신 귤', 'tangerine.jpg')


In [10]:
# 상품갯수를 추출
sql = "select count(*) from product"
cur.execute(sql)

# 레코드 갯수가 1개일 경우
count = cur.fetchone()
print("상품갯수 : ", count[0])

상품갯수 :  6


In [11]:
# 전체 레코드 삭제
sql = "delete from product"
cur.execute(sql)
conn.commit()

In [12]:
# 레코드를 일괄적으로 추가하는 작업
items = [
    (1, '레몬', 1500, '맛있는 레몬이 왔어요', 'lemon.jpg'),
    (2, '오렌지', 2500, '상큼한 오렌지 있어요', 'orange.jpg'),
    (3, '키위', 1000, '먹어도 키가 안크는 키위가 있어요', 'kiwi.jpg'),
    (4, '포도', 5000, '포도가 맛있다', 'grape.jpg'),
    (5, '딸기', 8000, '딸기도 좋아요', 'strawberry.jpg'),
    (6, '귤', 500, '제주도 대신 귤', 'tangerine.jpg')
]

sql = "insert into product values(:1, :2, :3, :4, :5)"
# 일괄적으로 레코드를 추가할 때는 레코드의 갯수를 지정해줘야 한다.
cur.bindarraysize = len(items)
cur.executemany(sql, items)
conn.commit()

In [14]:
cur.close()
conn.close()

In [15]:
conn = cx.connect("scott", "1234", "localhost:1521/xe")
## 커서 객체 생성
cur = conn.cursor()

## query(질의문)
sql = "select * from product"

## 데이터 프레임으로 불러오기

In [16]:
import pandas as pd
df = pd.read_sql(sql, con = conn)
print(df)

   PRODUCT_ID PRODUCT_NAME  PRICE         DESCRIPTION     PICTURE_URL
0           1           레몬   1500         맛있는 레몬이 왔어요       lemon.jpg
1           2          오렌지   2500         상큼한 오렌지 있어요      orange.jpg
2           3           키위   1000  먹어도 키가 안크는 키위가 있어요        kiwi.jpg
3           4           포도   5000             포도가 맛있다       grape.jpg
4           5           딸기   8000             딸기도 좋아요  strawberry.jpg
5           6            귤    500            제주도 대신 귤   tangerine.jpg


In [18]:
cur.execute(sql)
row = cur.fetchall()
colname = cur.description  # 컬럼명 추출
col = []

for i in colname:
    col.append(i[0])
    
product = pd.DataFrame(row, columns=col)
print(product)

   PRODUCT_ID PRODUCT_NAME  PRICE         DESCRIPTION     PICTURE_URL
0           1           레몬   1500         맛있는 레몬이 왔어요       lemon.jpg
1           2          오렌지   2500         상큼한 오렌지 있어요      orange.jpg
2           3           키위   1000  먹어도 키가 안크는 키위가 있어요        kiwi.jpg
3           4           포도   5000             포도가 맛있다       grape.jpg
4           5           딸기   8000             딸기도 좋아요  strawberry.jpg
5           6            귤    500            제주도 대신 귤   tangerine.jpg


# 회원관리 프로그램

- CRUD(CREATE, SELECT, UPDATE, INSERT, DELETE) 즉, DML(조작어)
- 테이블생성 Member(userid, userpwd, name), userid는 primary key 설정
- 주요기능 : 회원가입(등록), 회원수정, 회원정보출력, 회원삭제등

In [5]:
import cx_Oracle as cx

conn = cx.connect("scott", "1234", "localhost:1521/xe")

## member 테이블 조회

In [9]:
cursor = conn.cursor()
cursor.execute("select * from member")

# 모든 레코드를 Resultset 객체에 저장
rs = cursor.fetchall() # list
# print(type(rs))
# print(rs)
for member in rs:
    print(member)

('admin', '1234', '관리자')
('aaa', '1234', '홍길동')


In [22]:
rs[0]

('admin', 'admin', '관리자')

In [23]:
userid = rs[0][0]
print(userid)

admin


## 레코드 삽입

In [24]:
def insert(userid, userpass, username):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "insert into member values(:1, :2, :3)"
    
    cur = conn.cursor()
    cur.execute(sql, [userid, userpass, username])
    cur.close()
    conn.commit()
    conn.close()

In [25]:
print("회원가입할 회원의 정보를 입력하세요.")
userid = input('id = ')
userpass = input('pass = ')
username = input('name = ')

insert(userid, userpass, username)

회원가입할 회원의 정보를 입력하세요.
id = aaa
pass = 1234
name = 홍길동


## 레코드 수정

In [2]:
def update(userid, userpass, username):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "update member set userpass=:1, username=:2 where userid=:3"
    
    cur = conn.cursor()
    cur.execute(sql, [userpass, username, userid])
    
    cur.close()
    conn.commit()
    conn.close()

In [3]:
# id_check(userid)
def id_check(userid):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "select * from member"
    
    cur = conn.cursor()
    cur.execute(sql)
    rs = cur.fetchall()
    
    for row in rs:
        if row[0] == userid:
            return True
        
    return False

In [7]:
# 키보드를 통해 회원정보를 변경하는 작업
userid = input('id = ')
if id_check(userid):
    userpass = input('pass = ')
    username = input('name = ')
    update(userid, userpass, username)
else:
    print(f"{userid} 회원정보가 존재하지 않습니다.")

id = aaa
pass = aaa
name = 장보고


# 최종 프로그램

## DB 연동

In [10]:
import cx_Oracle as cx

conn = cx.connect("scott", "1234", "localhost:1521/xe")

## 기능별 함수 구현

## main processor

In [11]:
# 모듈 포함
from dbTest import *

while True:
    print("\n== 회원관리 프로그램 ==")
    print("1.회원등록")
    print("2.회원수정")
    print("3.회원삭제")
    print("4.회원출력")
    print("5.작업종료")
    menu = int(input('메뉴 = '))
    
    if menu == 1:
        userid = input('id = ')
        userpass = input('pass = ')
        username = input('name = ')
        insert(userid, userpass, username)
        
    elif menu == 2:
        userid = input('id = ')
        if id_check(userid) == 0:
            print(f"{userid} 회원정보가 존재하지 않습니다.")
        else:
            print("수정할 정보를 입력하세요")
            userpass = input('pass = ')
            username = input('name = ')
            update(userid, userpass, username)
    elif menu == 3:
        userid = input('id = ')
        if id_check(userid) == 0:
            print(f"{userid} 회원정보가 존재하지 않습니다.")
        else:
            delete(userid)
    elif menu == 4:
        listMember()
    elif menu == 5:
        close()
        break
    else:
        print("메뉴의 선택이 올바르지 않습니다.")
        
print("프로그램을 종료합니다.")


== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 4
('admin', '1234', '관리자')
('aaa', '1234', '홍길동')

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 1
id = bbb
pass = 3333
name = 강아지
회원등록이 완료되었습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 4
('admin', '1234', '관리자')
('aaa', '1234', '홍길동')
('bbb', '3333', '강아지')

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 2
id = ccc
ccc 회원정보가 존재하지 않습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 2
id = bbb
수정할 정보를 입력하세요
pass = 1234
name = 강감찬
회원정보가 수정되었습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 4
('admin', '1234', '관리자')
('aaa', '1234', '홍길동')
('bbb', '1234', '강감찬')

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 3
id = ddd
ddd 회원정보가 존재하지 않습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 3
id = bbb

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 4
('admin', '1234', '관리자')
('aaa', '1234', '홍길동')

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴